<a href="https://colab.research.google.com/github/GabsLUZ/Trabalho_Final_IA/blob/main/Teste_Avaliza%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install scikit-plot

In [ ]:
pip install -U scikit-learn

In [ ]:
# Manipulacao de dados
import numpy as np
import pandas as pd

# Visualizacao de dados
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

# Machine learning
import scikitplot as skplt
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# cross validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_validate, KFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# Selecao de caracteristicas e encoders
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Metricas de avaliacao
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, average_precision_score, RocCurveDisplay

# remover warning
import warnings
warnings.filterwarnings('ignore')

A biblioteca  plot_roc_curve não está mais disponivel na versão atual do python então foi utilizado o RocCurveDisplay

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/trabalho IA/covid_2022_cleaned.csv", sep=';')
df_ = pd.read_csv("/content/drive/MyDrive/trabalho IA/covid_2022_cleaned_raca.csv", sep=';')

In [ ]:
df.head()

In [ ]:
def print_df_dimensions(df):
    print(f'O DataFrame tem {df.shape[0]} linhas e {df.shape[1]} colunas.')

In [ ]:
print_df_dimensions(df)

In [ ]:
print_df_dimensions(df_)

In [ ]:
df.describe(include='O')

In [ ]:
df[df['EVOLUCAO'] == 1].describe(include='O')

1. Feature Encoding
Como modelos de machine learning performam algebra linear em matrizes, precisamos que todas as variáveis estejam como valores numericos. O processo de conversão de variaveis categoricas em numéricas é chamado de Encoding. Existem algumas abordagens para converter os dados categóricos em numéricos, para que o modelo possa entendê-los. Veremos os dois métodos: Label encoding e One-Hot encoding.

Para nosso contexto, se utilizássemos o **One-hot encoding**, aplicaria apenas na 'FAIXAETARIA', pois é a única que possui mais de 2 valores possíveis. As demais variáveis são dicotômicas, e por isso, caso utilizássemos o one-hot, na prática este seria o **label encoding**. Então, vamos utilizar diretamente este último.

In [ ]:
df['FAIXAETARIA'].value_counts()

In [ ]:
# label encoding
df['FAIXAETARIA'] = df['FAIXAETARIA'].map({'<1': 0, 
                       '01 a 04': 1, 
                       '05 a 09': 2,
                       '10 a 14': 3, 
                       '15 a 19': 4,
                       '20 a 29': 5, 
                       '30 a 39': 6, 
                       '40 a 49': 7, 
                       '50 a 59': 8, 
                       '60 a 69': 9,
                       '70 a 79': 10, 
                       '80 e mais': 11})

In [ ]:
# label encoding raça
df_['FAIXAETARIA'] = df_ ['FAIXAETARIA'].map({'<1': 0, 
                       '01 a 04': 1, 
                       '05 a 09': 2,
                       '10 a 14': 3, 
                       '15 a 19': 4,
                       '20 a 29': 5, 
                       '30 a 39': 6, 
                       '40 a 49': 7, 
                       '50 a 59': 8, 
                       '60 a 69': 9,
                       '70 a 79': 10, 
                       '80 e mais': 11})

In [ ]:
df['FAIXAETARIA'].value_counts()

In [ ]:
# LabelEncoder
le = LabelEncoder()

# apply "le.fit_transform"
df = df.apply(le.fit_transform)
df

In [ ]:
# raça
# apply "le.fit_transform"
df_ = df_.apply(le.fit_transform)

In [ ]:
# salvar no disco
df.to_csv("/content/drive/MyDrive/trabalho IA/covid_2022_prepared.csv", sep=';', index=False)
df_.to_csv("/content/drive/MyDrive/trabalho IA/covid_2022_prepared_raça.csv", sep=";", index=False)

In [ ]:
# sem raca
X_val = df.drop(['EVOLUCAO'], axis=1)
y_val = df['EVOLUCAO']

# com raca/cor
X_val_ = df_.drop(['EVOLUCAO'], axis=1)
y_val_ = df_['EVOLUCAO']

# Validacao
X, X_test_val, y, y_test_val = train_test_split(X_val, y_val, stratify=y_val, test_size=0.3, random_state=777)
X_, X_test_val_, y_, y_test_val_ = train_test_split(X_val_, y_val_, stratify=y_val_, test_size=0.3, random_state=777)

# treino e teste
y.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)
y_.reset_index(drop=True, inplace=True)
X_.reset_index(drop=True, inplace=True)

In [ ]:
y.value_counts()

In [ ]:
y_.value_counts()

In [ ]:
from sklearn.metrics import precision_recall_curve
import scikitplot as skplt
from sklearn.metrics import confusion_matrix

def print_metric(results, name):
    media = np.mean(results)
    desvio_padrao = np.std(results)
    print(f"{name} ", end='')
    print("médio: %.3f | intervalo: [%.3f, %.3f]" % ((media), (media - 2 * desvio_padrao), (media + 2 * desvio_padrao)))

In [ ]:
import scikitplot as skplt
# initial parameters
SEED = 777
K_FOLD = 5


# definicao dos modelos
models = [
          RandomForestClassifier(random_state=SEED), 
          ]

# nome dos modelos
verbose_models = ['Floresta aleatória']

# figuras para plots
# plt.rcParams.update({'font.size': 12})
# fig1, ax1 = plt.subplots(figsize=(12, 6))
# fig2, ax2 = plt.subplots(figsize=(12, 6))

for name, model in zip(verbose_models, models):
    # kfold
    kf = StratifiedKFold(n_splits=K_FOLD, shuffle=True, random_state=SEED)

    print("===============")
    print(name + " - Sem raça/cor")
    # percorre cada fold
    auc_roc = []
    precision = []
    recall = []
    f1score = []
    
    for train_index, test_index in kf.split(X,y):
        # divisao treino e teste
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # treino e predicao
        y_pred = model.fit(X_train, y_train).predict(X_test)

        # computacao da metrica
        auc_roc.append(roc_auc_score(y_test, y_pred))
        precision.append(precision_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        f1score.append(f1_score(y_test, y_pred))

    print_metric(auc_roc, 'AUC-ROC')
    print_metric(precision, 'PRECISÃO')
    print_metric(recall, 'REVOCAÇÃO')
    print_metric(f1score, 'F1 SCORE')
    print(confusion_matrix(y_test, y_pred))

    #skplt.metrics.plot_roc(X_test, y_test)

    # plot_precision_recall_curve(model, X_test, y_test, ax=ax2, name=name)
    print("===============")

    # kfold
    kf = StratifiedKFold(n_splits=K_FOLD, shuffle=True, random_state=SEED)
    print("===============")
    name = name + " - Com raça/cor"
    print(name)
    # percorre cada fold
    auc_roc = []
    precision = []
    recall = []
    f1score = []
    
    for train_index, test_index in kf.split(X_,y_):
        # divisao treino e teste
        X_train, X_test = X_.loc[train_index], X_.loc[test_index]
        y_train, y_test = y_[train_index], y_[test_index]

        # treino e predicao
        y_pred = model.fit(X_train, y_train).predict(X_test)

        # computacao da metrica
        auc_roc.append(roc_auc_score(y_test, y_pred))
        precision.append(precision_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        f1score.append(f1_score(y_test, y_pred))

    print_metric(auc_roc, 'AUC-ROC')
    print_metric(precision, 'PRECISÃO')
    print_metric(recall, 'REVOCAÇÃO')
    print_metric(f1score, 'F1 SCORE')
    print(confusion_matrix(y_test, y_pred))
    # plot_roc_curve(model, X_test, y_test, ax=ax1, name=name)
    # plot_precision_recall_curve(model, X_test, y_test, ax=ax2, name=name)
    print("===============")

#_ = ax1.set_title('Curva ROC')
#_ = ax2.set_title('Curva Precisão-Sensibilidade')
#plt.show()     

In [ ]:
from sklearn.model_selection import cross_val_score
# balanceamento

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

In [ ]:
print("Sem raça/cor")
rus = RandomUnderSampler(random_state=777)
# kfold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)

imba_pipeline = make_pipeline(rus, RandomForestClassifier(random_state=SEED))
score = cross_val_score(imba_pipeline, X, y, scoring='roc_auc', cv=kf)
print_metric(score, 'ROC AUC')

In [ ]:
print("Com raça/cor")
rus = RandomUnderSampler(random_state=777)
# kfold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)

imba_pipeline = make_pipeline(rus, RandomForestClassifier(random_state=SEED))
score = cross_val_score(imba_pipeline, X_, y_, scoring='roc_auc', cv=kf)
print_metric(score, 'ROC AUC')